In [12]:
import sys
import os

project_root = os.path.abspath('..')
if project_root not in sys.path:
    sys.path.append(project_root)
 
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style()
plt.rcParams["figure.figsize"] = (12, 6)
pd.set_option("display.max_columns", None)

from src.features import build_features

# Define dictionaries for column names and types
PROCEDURE_COLUMNS = {f"PROCEDIMIENTO{i}": str for i in range(1, 31)}
SERVICE_COLUMNS = {
    "SERVICIOINGRESO": 0,
    "SERVICIOTRASLADO1": 1,
    "SERVICIOTRASLADO2": 2,
    "SERVICIOTRASLADO3": 3,
    "SERVICIOTRASLADO4": 4,
    "SERVICIOTRASLADO5": 5,
    "SERVICIOTRASLADO6": 6,
    "SERVICIOTRASLADO7": 7,
    "SERVICIOTRASLADO8": 8,
    "SERVICIOTRASLADO9": 9,
    "SERVICIOALTA": 10,
}
DATE_COLUMNS = {
    "FECHA_INGRESO": 0,
    "FECHATRASLADO1": 1,
    "FECHATRASLADO2": 2,
    "FECHATRASLADO3": 3,
    "FECHATRASLADO4": 4,
    "FECHATRASLADO5": 5,
    "FECHATRASLADO6": 6,
    "FECHATRASLADO7": 7,
    "FECHATRASLADO8": 8,
    "FECHATRASLADO9": 9,
    "FECHAALTA": 10,
}

CAMBIO_CAMAS = {
    "AREA MEDICA ADULTO CUIDADOS BASICOS": "Basicos",
    "AREA MEDICA ADULTO CUIDADOS MEDIOS": "Medias",
    "AREA MEDICO-QUIRURGICO CUIDADOS MEDIOS": "Medias",
    "AREA MÉDICA": "Medias",
    "AREA QUIRÚRGICA": "Medias",
    "CIRUGÍA CARDIOVASCULAR": "Medias",
    "UNIDAD DE CUIDADOS INTENSIVOS (UCI) (INDIFERENCIADO)": "UCI",
    "UNIDAD DE CUIDADOS INTENSIVOS ADULTO": "UCI",
    "UNIDAD DE CUIDADOS INTENSIVOS CARDIOLOGÍA": "UCI",
    "UNIDAD DE RECUPERACIÓN DE PABELLONES (CENTRAL Y CMA)": "AMBULATORIO",
    "UNIDAD DE TRATAMIENTO INTERMEDIO (UTI) (INDIFERENCIADO) ADULTO": "UTI",
    "UNIDAD DE TRATAMIENTO INTERMEDIO CARDIOVASCULAR": "UTI",
    "UNIDAD DE TRATAMIENTO INTERMEDIO CIRUGÍA ADULTO": "UTI",
    "UNIDAD DE TRATAMIENTO INTERMEDIO MEDICINA ADULTO": "UTI",
    "UNIDAD DE TRATAMIENTOS INTERMEDIOS MEDICINA": "UTI",
}

In [14]:
DIAGS_HOSPITALIZADOS = [
    "A152",
    "B441",
    "C34N",
    "C341",
    "C342",
    "C343",
    "C780",
    "C782",
    "D381",
    "C381",
    "E848",
    "Enfermedad Coronaria",
    "I251",
    "Enfermedad tromboembólica crónica pulmonar",
    "I269",
    "I051",
    "I052",
    "I080",
    "I081",
    "I232",
    "I330",
    "I340",
    "I350",
    "I352",
    "I351",
    "I361",
    "I420",
    "I421",
    "I495",
    "I710",
    "I711",
    "I712",
    "I715",
    "J679",
    "J841",
    "J848",
    "J955",
    "M051",
    "M348",
    "Q676",
    "Q677",
    "QXXX",
    "Q201",
    "Q202",
    "Q203",
    "Q204",
    "Q205",
    "Q206",
    "Q208",
    "Q209",
    "Q210",
    "Q211",
    "Q212",
    "Q213",
    "Q214",
    "Q218",
    "Q220",
    "Q221",
    "Q222",
    "Q223",
    "Q224",
    "Q225",
    "Q228",
    "Q230",
    "Q231",
    "Q233",
    "Q240",
    "Q241",
    "Q244",
    "Q245",
    "Q246",
    "Q248",
    "Q249",
    "Q250",
    "Q251",
    "Q253",
    "Q254",
    "Q255",
    "Q256",
    "Q257",
    "Q258",
    "Q259",
    "Q264",
    "Q268",
    "Q272",
    "Q273",
    "Q288",
    "Q289",
    "Q311",
    "Q320",
    "Q321",
    "Q330",
    "Q331",
    "Q332",
    "Q334",
    "Q338",
    "Q341",
    "Q348",
    "Q678",
    "Q765",
    "Q766",
    "Q767",
    "Q768",
    "Q769",
    "Q780",
    "Q790",
    "Q798",
    "Q839",
    "Q850",
    "Q858",
    "Q859",
    "Q874",
    "Q893",
    "Z450",
    "Z941",
    "Z942",
]

In [15]:
# Lee la base de datos GRD del INT
df_torax = pd.read_csv("../data/processed/df_procesada_112103.csv", sep=";")
df_torax = df_torax.query("TIPO_ACTIVIDAD == 'HOSPITALIZACIÓN'")
df_torax = df_torax.reset_index(drop=True).reset_index(names="id_egreso")
df_torax["DIAGNOSTICO1"] = (
    df_torax["DIAGNOSTICO1"].str.replace(".", "", regex=False).str.ljust(4, "X")
)

# Deja solamente los diagnosticos relevantes
df_torax_hospitalizados = df_torax.query("DIAGNOSTICO1.isin(@DIAGS_HOSPITALIZADOS)")

C:\Users\jarojas\AppData\Local\Temp\ipykernel_14500\1368003325.py:2: DtypeWarning: Columns (85,86,88,90) have mixed types. Specify dtype option on import or set low_memory=False.
  df_torax = pd.read_csv("../data/processed/df_procesada_112103.csv", sep=";")


In [16]:
# Convierte los traslados y sus fechas a formato long
viaje_paciente = build_features.crear_viaje_paciente(df_torax, SERVICE_COLUMNS, DATE_COLUMNS)

# Procesa el viaje del paciente, ordenando las fechas de traslado y eliminando las que no se imputan
viaje_paciente = build_features.procesar_viaje_paciente(
    viaje_paciente, SERVICE_COLUMNS, ["id_egreso", "tipo_servicio"], CAMBIO_CAMAS
)

# Calcula los dias de estadia por cada uno de los servicios
viaje_paciente = build_features.procesar_duracion_estadia(viaje_paciente)

In [17]:
DIAGS_CANCER = ["C341", "C342", "C343", "C381"]
DIAGS_ATEROSCLEROTICA = ["I251"]

# Crea un nuevo viaje a modificar
viaje_paciente_modificado = viaje_paciente.copy()

# Modifica registros de cancer entre 2020 a 2023 y Medias
mask_registros_cancer_a_modificar = (
    viaje_paciente_modificado["ANIO_EGRESO"].isin([2020, 2021, 2022, 2023])
) & (
    viaje_paciente_modificado["DIAGNOSTICO1"].isin(DIAGS_CANCER)
    & (viaje_paciente_modificado["servicio"] == "Medias")
)

# Modifica la estadia de los diags de cancer
viaje_paciente_modificado.loc[mask_registros_cancer_a_modificar, "duracion_estadia"] = (
    viaje_paciente_modificado[mask_registros_cancer_a_modificar]["duracion_estadia"] * 0.2
)

In [20]:
# Obtiene el registro de coronarias medias
mask_registros_coronarias_medias_a_modificar = (
    (viaje_paciente_modificado["ANIO_EGRESO"].isin([2020, 2021, 2022, 2023]))
    & (viaje_paciente_modificado["DIAGNOSTICO1"].isin(DIAGS_ATEROSCLEROTICA))
    & (viaje_paciente_modificado["servicio"] == "Medias")
)

# obtiene el registro de coronarias UTI
mask_registros_coronarias_uti_a_modificar = (
    (viaje_paciente_modificado["ANIO_EGRESO"].isin([2020, 2021, 2022, 2023]))
    & (viaje_paciente_modificado["DIAGNOSTICO1"].isin(DIAGS_ATEROSCLEROTICA))
    & (viaje_paciente_modificado["servicio"] == "UTI")
)

# Modifica la estadia de los diags de coronarias Medias (Saca el 20%)
viaje_paciente_modificado.loc[mask_registros_coronarias_medias_a_modificar, "duracion_estadia"] = (
    viaje_paciente_modificado[mask_registros_coronarias_medias_a_modificar]["duracion_estadia"]
    * 0.2
)
# Modifica la estadia de las camas UTI (Agrega el 80% de las camas medias, hard codeado)
viaje_paciente_modificado.loc[mask_registros_coronarias_uti_a_modificar, "duracion_estadia"] = (
    viaje_paciente_modificado[mask_registros_coronarias_uti_a_modificar]["duracion_estadia"]
    * 2.727884615384615
)

In [ ]:
def unir_resumen_a_cie(resumen, diccionario_cie):
    resumen = resumen.copy()

    # Cambia la columna de diagnostico
    resumen["DIAGNOSTICO1"] = (
        resumen["DIAGNOSTICO1"].str.replace(".", "", regex=False).str.ljust(4, "X")
    )

    # Pone como Indice el diagnostico CIE
    resumen = resumen.set_index("DIAGNOSTICO1")

    # Une el diccionario CIE al resumen
    resumen = resumen.join(diccionario_cie).reset_index()
    resumen = resumen.sort_values(["ANIO_EGRESO", "DIAGNOSTICO1"])
    
    return resumen

In [ ]:
# Obtiene el resumen de dias de estada totales por cada uno de los diags
resumen_dias_de_estada = build_features.procesar_resumen_dias_estada(viaje_paciente)
resumen_dias_de_estada_mod = build_features.procesar_resumen_dias_estada(viaje_paciente_modificado)

# Une los resumenes al diccionario CIE
resumen_dias_de_estada = unir_resumen_a_cie(resumen_dias_de_estada, diccionario_cie)
resumen_dias_de_estada_mod = unir_resumen_a_cie(resumen_dias_de_estada_mod, diccionario_cie)

NameError: name 'diccionario_cie' is not defined

In [ ]:
resultados = []
filtros_a_realizar = [DIAGS_HOSPITALIZADOS, DIAGS_CANCER_SIN_PUNTO]
for lista_diags in filtros_a_realizar:
    resumen_DE_filtrado = resumen_dias_de_estada.query("DIAGNOSTICO1.isin(@lista_diags)")
    resumen_DE_filtrado_modificado = resumen_dias_de_estada_mod.query(
        "DIAGNOSTICO1.isin(@lista_diags)"
    )
    resultados.append(resumen_DE_filtrado, resumen_DE_filtrado_modificado)

TypeError: list.append() takes exactly one argument (2 given)

In [ ]:
# Filtra el resumen solamente para diags hospitalizados
resumen_dias_de_estada_hosp = resumen_dias_de_estada[
    resumen_dias_de_estada["DIAGNOSTICO1"].isin(DIAGS_HOSPITALIZADOS)
]

# Filtra el resumen de dias de estada por diagnosticos relevantes
resumen_dias_de_estada_relevantes = resumen_dias_de_estada[
    resumen_dias_de_estada["DIAGNOSTICO1"].isin(TODOS_LOS_DIAGS_RELEVANTES)
]

# Filtra el resumen solamente para cancer
resumen_dias_de_estada_cancer = resumen_dias_de_estada[
    resumen_dias_de_estada["DIAGNOSTICO1"].isin(DIAGS_CANCER_SIN_PUNTO)
]

# Filtra el resumen solamente para diags hospitalizados
resumen_dias_de_estada_hosp_mod = resumen_dias_de_estada[
    resumen_dias_de_estada["DIAGNOSTICO1"].isin(DIAGS_HOSPITALIZADOS)
]

# Filtra el resumen de dias de estada por diagnosticos relevantes
resumen_dias_de_estada_relevantes_mod = resumen_dias_de_estada[
    resumen_dias_de_estada["DIAGNOSTICO1"].isin(TODOS_LOS_DIAGS_RELEVANTES)
]

# Filtra el resumen solamente para cancer
resumen_dias_de_estada_cancer_mod = resumen_dias_de_estada[
    resumen_dias_de_estada["DIAGNOSTICO1"].isin(DIAGS_CANCER_SIN_PUNTO)
]
    

# Elimina columna innecesaria
resumen_dias_de_estada = resumen_dias_de_estada.set_index(["codigo_descr"]).drop(
    columns="DIAGNOSTICO1"
)

resumen_dias_de_estada_hosp = resumen_dias_de_estada_hosp.set_index(["codigo_descr"]).drop(
    columns="DIAGNOSTICO1"
)

resumen_dias_de_estada_relevantes = resumen_dias_de_estada_relevantes.set_index(
    ["codigo_descr"]
).drop(columns="DIAGNOSTICO1")

In [ ]:
with pd.ExcelWriter("../data/interim/resumen_ocupacion_por_diagnostico_DE.xlsx") as file:
    resumen_dias_de_estada.to_excel(file, sheet_name="todos_los_diags")
    resumen_dias_de_estada_hosp.to_excel(file, sheet_name="diags_hosp")
    resumen_dias_de_estada_relevantes.to_excel(file, sheet_name="diags_relevantes")

## Resumen Metricas

In [ ]:
resumen_egresos = df_torax.groupby("ANIO_EGRESO").size()
resumen_egresos_relevantes = df_torax_hospitalizados.groupby("ANIO_EGRESO").size()
resumen_egresos_totales = pd.DataFrame(
    {"total_egresos": resumen_egresos, "egresos_relevantes": resumen_egresos_relevantes}
)
resumen_egresos_totales["porcentaje_de_egresos"] = (
    resumen_egresos_totales["egresos_relevantes"] / resumen_egresos_totales["total_egresos"]
)

In [ ]:
resumen_egresos_totales

,total_egresos,egresos_relevantes,porcentaje_de_egresos
ANIO_EGRESO,,,
2019,5434,NaN,NaN
2020,2850,NaN,NaN
2021,3086,NaN,NaN
2022,3217,NaN,NaN
2023,3594,NaN,NaN


In [ ]:
resumen_dias_estada_totales = df_torax.groupby("ANIO_EGRESO")["ESTANCIA"].sum()
resumen_dias_estada_totales_relevantes = df_torax_hospitalizados.groupby("ANIO_EGRESO")[
    "ESTANCIA"
].sum()

resumen_dias_estada_totales = pd.DataFrame(
    {
        "total_dias_estada": resumen_dias_estada_totales,
        "dias_estada_relevantes": resumen_dias_estada_totales_relevantes,
    }
)
resumen_dias_estada_totales["porcentaje_de_dias_estada"] = (
    resumen_dias_estada_totales["dias_estada_relevantes"]
    / resumen_dias_estada_totales["total_dias_estada"]
)

In [ ]:
resumen_dias_estada_totales

,total_dias_estada,dias_estada_relevantes,porcentaje_de_dias_estada
ANIO_EGRESO,,,
2019,42916,NaN,NaN
2020,31375,NaN,NaN
2021,36679,NaN,NaN
2022,33016,NaN,NaN
2023,32385,NaN,NaN


In [ ]:
resumen_dias_de_estada["dias_estada_totales"].reset_index().groupby("ANIO_EGRESO").sum()

KeyError: 'dias_estada_totales'